# Imports and data loading

In [1]:
%run Imports.ipynb
name = 'Kred' # Choose Kred or Mone

param_grid = {
    'learning_rate': [0.001, 0.005, 0.01, 0.05],
    'max_leaves': [3, 5, 10, 15, 20, 32],
    'min_samples_leaf': [2, 5, 10, 15, 20],
    'interactions': [5, 10, 15]
}

/home/RDC/fliegebe/.conda/envs/Work/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
key_featsubgroups = pd.read_pickle('../../pickle/2_FS/' + name + '/key_featsubgroups.pkl')
df = pd.read_pickle('../../pickle/2_FS/' + name + '/2_df_new_.pkl')

In [3]:
print(df[target].value_counts()/df.shape[0])
print('df_shape: ', df.shape)

arrears
1   0.646
0   0.354
Name: count, dtype: float64
df_shape:  (129457, 418)


In [4]:
def split_data_4(df, key_featsubgroups=key_featsubgroups, target=target, test_size=0.2, random_state=42):
    """
    Splits the dataset into training and testing sets while excluding demographic features.

    Parameters:
    df (DataFrame): The dataset containing features and target variable.
    key_featsubgroups (DataFrame): A mapping of feature subgroups.
    target (str): The name of the target variable.
    test_size (float, optional): The proportion of the dataset to allocate for testing. Default is 0.2.
    random_state (int, optional): Random seed for reproducibility. Default is 42.

    Returns:
    tuple: X_train, X_test, y_train, y_test (training and testing datasets)
    """

    # Extract demographic features
    demo_feat = key_featsubgroups.loc[key_featsubgroups['subgroup'] == 'demo', 'list_features'].values[0]
    print("Demographic Features:", demo_feat)

    # Separate features (X) and target variable (y), excluding demographic features
    X = df.drop(columns=[target] + demo_feat)
    y = df[target]

    # Split the dataset into training (80%) and testing (20%) sets
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state
    )

    # Print dataset shapes
    print(f"Training Features Shape: {X_train.shape}")
    print(f"Training Labels Shape: {y_train.shape}")
    print(f"Testing Features Shape: {X_test.shape}")
    print(f"Testing Labels Shape: {y_test.shape}")

    return X_train, X_test, y_train, y_test

In [5]:
with open('../../pickle/Monedo_5/wo_optuna/best_params.pkl', 'rb') as f:
    best_params = pickle.load(f)

In [6]:
best_params

{'interactions': 15,
 'learning_rate': 0.01,
 'max_leaves': 10,
 'min_samples_leaf': 2}

In [7]:
# with open('../pickle/3_Model/xgb_default_params.pkl', 'rb') as f:
#     default_params = pickle.load(f)

# Load results_dict_updated from the pickle file
with open('../../pickle/4_Model_Optuna/results_dict_updated.pkl', 'rb') as f:
    results_dict = pickle.load(f)

In [8]:
with open('../../pickle/4_Model_Optuna/results_dict_updated_5.pkl', 'rb') as f:
    rd = pickle.load(f)

# Task 2

In [9]:
def train_ebm(df, best_params, results_dict):
    start_time = time.time()
    print("\nStarting XGBoost model training and evaluation...")

    # Split dataset
    X_train, X_test, y_train, y_test = split_data_4(df)

    # Initialize and train EBM with best parameters
    ebm =  ExplainableBoostingClassifier(**best_params)
    ebm.fit(X_train, y_train)

    # Perform cross-validation with CV=5
    y_train_pred = cross_val_predict(ebm, X_train, y_train, cv=5)

    # Make predictions on test set
    y_test_pred = ebm.predict(X_test)

    # Compute confusion matrices
    cfm_train = confusion_matrix(y_train, y_train_pred)
    cfm_test = confusion_matrix(y_test, y_test_pred)

    # Compute accuracy scores
    accs_train = accuracy_score(y_train, y_train_pred)
    accs_test = accuracy_score(y_test, y_test_pred)

    # Compute F1-scores for both classes (0 and 1)
    f1s_train_p1 = f1_score(y_train, y_train_pred, pos_label=1)
    f1s_train_p0 = f1_score(y_train, y_train_pred, pos_label=0)
    f1s_test_p1 = f1_score(y_test, y_test_pred, pos_label=1)
    f1s_test_p0 = f1_score(y_test, y_test_pred, pos_label=0)

    # Compute ROC-AUC score for test data
    test_ras = roc_auc_score(y_test, ebm.predict_proba(X_test)[:, 1])

    # Calculate total runtime in minutes
    total_time = (time.time() - start_time) / 60
    print(f"EBM Model training completed in {total_time:.2f} minutes")

    # Store computed values in results dictionary
    results_dict["ebm_ben"] = {
        "classifier": deepcopy(ebm),
        "cfm_train": cfm_train,
        "cfm_test": cfm_test,
        "train_accuracy": accs_train,
        "test_accuracy": accs_test,
        "train F1-score label 1": f1s_train_p1,
        "train F1-score label 0": f1s_train_p0,
        "test F1-score label 1": f1s_test_p1,
        "test F1-score label 0": f1s_test_p0,
        "test roc auc score": test_ras,
        "best_params": best_params,
        "time_m": total_time
    }

    return results_dict

In [10]:
rd = train_ebm(df, best_params, rd)


Starting XGBoost model training and evaluation...
Demographic Features: ['clientdata.demo.gender', 'clientdata.demo.age_year', 'clientdata.demo.age_month', 'clientdata.demo.children', 'clientdata.demo.children_singleparent', 'clientdata.demo.maritalstatus_expand_SINGLE', 'clientdata.demo.maritalstatus_expand_MARRIED', 'clientdata.demo.maritalstatus_expand_DIVORCED', 'clientdata.demo.maritalstatus_expand_WIDOWED', 'clientdata.demo.maritalstatus_expand_newvalue', 'clientdata.demo.maritalstatus_woe']
Training Features Shape: (103565, 406)
Training Labels Shape: (103565,)
Testing Features Shape: (25892, 406)
Testing Labels Shape: (25892,)
EBM Model training completed in 12.66 minutes


In [12]:
with open('../../pickle/4_Model_Optuna/results_dict_updated_5.pkl', 'wb') as f:
    pickle.dump(rd, f, pickle.HIGHEST_PROTOCOL)

In [7]:
X_train, X_test, y_train, y_test = split_data_4(df)

Demographic Features: ['clientdata.demo.gender', 'clientdata.demo.age_year', 'clientdata.demo.age_month', 'clientdata.demo.children', 'clientdata.demo.children_singleparent', 'clientdata.demo.maritalstatus_expand_SINGLE', 'clientdata.demo.maritalstatus_expand_MARRIED', 'clientdata.demo.maritalstatus_expand_DIVORCED', 'clientdata.demo.maritalstatus_expand_WIDOWED', 'clientdata.demo.maritalstatus_expand_newvalue', 'clientdata.demo.maritalstatus_woe']
Training Features Shape: (103565, 406)
Training Labels Shape: (103565,)
Testing Features Shape: (25892, 406)
Testing Labels Shape: (25892,)


In [8]:
# train model with best params
from interpret.glassbox import ExplainableBoostingClassifier

ebm_best = ExplainableBoostingClassifier(**best_params)
ebm_best.fit(X_train, y_train)

ExplainableBoostingClassifier(interactions=15, learning_rate=0.01,
                              max_leaves=10, min_samples_leaf=2)

In [9]:
# predict test data
y_test_pred  = ebm_best.predict(X_test)
y_test_proba = ebm_best.predict_proba(X_test)[:, 1]

In [11]:
# Extract feature importance score from trained model using permutation-based feature importance scoring
from sklearn.inspection import permutation_importance

perm_imp = permutation_importance(
    ebm_best, X_test, y_test,
    n_repeats=30, random_state=0, n_jobs=-1
)
feature_importances = perm_imp.importances_mean

In [12]:
# Store test set predictions and feature importance scores
ebm_best_results = {
    "y_test_pred":         y_test_pred,
    "y_test_proba":        y_test_proba,
    "feature_importances": feature_importances
}

# please change the path to your desired location
with open("../../pickle//Monedo_5/wo_optuna/ebm_best_results.pkl", "wb") as f:
    pickle.dump(ebm_best_results, f, protocol=pickle.HIGHEST_PROTOCOL)